In [0]:
#| default_exp preprocessing.lesson_23

# Lesson 23 from fastai
> Lesson23 unet

In [0]:
#| hide
%load_ext autoreload
%autoreload 2

In [0]:
#| export
from cv_tools.core import *


In [0]:
#| hide
import nbdev; nbdev.nbdev_export('06_preprocessing.lesson_23.ipynb')